In [1]:
import os
import pathlib
import shutil
import json
import polars as pl
from pprint import pprint
from tqdm import tqdm

pl.Config.load_from_file("pl_config.json")
pl.set_random_seed(999)

## 다운로드

다운받을 데이터셋 정보 조회

In [2]:
!aihubshell -mode l -datasetkey 653

aihubshell version 24.01.29 v0.3
Fetching file tree structure...
The contents are encoded in UTF-8 including Korean characters. 
If the following contents are not output normally, 
Please modify the character information of the OS. 
=================공지사항=================== 

    └─029.대규모 구매도서 기반 한국어 말뭉치 데이터
        ├─01.데이터
        │  ├─1.Training
        │  │  ├─원천데이터
        │  │  │  ├─TS_600.zip | 38 MB | 174654
        │  │  │  ├─TS_320.zip | 896 MB | 174630
        │  │  │  ├─TS_360.zip | 176 MB | 174631
        │  │  │  ├─TS_370.zip | 125 MB | 174632
        │  │  │  ├─TS_380.zip | 42 MB | 174633
        │  │  │  ├─TS_390.zip | 27 MB | 174634
        │  │  │  ├─TS_400.zip | 41 MB | 174635
        │  │  │  ├─TS_410.zip | 17 MB | 174636
        │  │  │  ├─TS_420.zip | 25 MB | 174637
        │  │  │  ├─TS_430.zip | 6 MB | 174638
        │  │  │  ├─TS_440.zip | 16 MB | 174639
        │  │  │  ├─TS_450.zip | 10 MB | 174640
        │  │  │  ├─TS_470.zip | 34 MB | 174641
        │  │  

AI Hub 데이터셋 소개란에 기재된 [개요](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=data&dataSetSn=653)를 살펴보고 내린 결정:

* 책의 한 문단에 속하는 여러 문장들을 하나의 문서로 봐야한다. 원본 데이터에서는 그런 구조를 파싱하기 어려웠기 때문에 라벨링데이터를 활용한다.
* 8로 시작하는 문학 텍스트는 아웃라이어같은 인물명을 가진 인물들 간 대화로 구성된 텍스트의 비중이 크다. 언어모델을 학습할 때 이처럼 특수한 데이터가 포함되는 것이 일반화 성능을 낮출 수 있으므로 제외한다.
* 원문 텍스트(original_text)에는 한자 등 특수문자가 다수 포함된 텍스트 비중이 적지 않아서 이를 가공해 만든 문장 텍스트(text)를 활용한다.

다운받은 데이터 디렉토리 및 결과 디렉토리 정의

In [3]:
archive_dir = [path for path in pathlib.Path(os.getcwd()).glob("*구매도서*") if path.suffix != ".ipynb"][0]
result_dir = pathlib.Path(f"{os.getcwd()}/aihub-datasets")
result_dir = result_dir.joinpath(archive_dir.name)
if result_dir.exists():
    os.system(f"rm -rf {result_dir}")
result_dir.mkdir(parents=True, exist_ok=True)

## 압축 해제 및 EDA

원천데이터 압축파일 경로 정의

In [4]:
archive_paths = []
for walk in os.walk(archive_dir):
    current_path, folders, files = walk
    for file_name in files:
        if file_name.endswith(".zip") and current_path.endswith("라벨링데이터"):
            archive_paths.append(f"{current_path}/{file_name}")

압축 해제

In [5]:
for archive_path in archive_paths:
    data_dir = pathlib.Path(archive_path.strip(".zip"))
    shutil.unpack_archive(
        filename=archive_path,
        extract_dir=data_dir,
        format="zip",
    )

각 데이터는 json 형태로 저장되어 있다. 파일 하나가 `paragraphs` 필드 밑에 하나 이상의 문장을 달고 있는 양상을 보인다.

In [6]:
data_files = list((archive_dir / "01.데이터").glob("*/라벨링데이터/[TV]L_[0-79]*/*TEXT*.json"))
data_files += list((archive_dir / "01.데이터").glob("*/라벨링데이터/[TV]L_unscramble/[0-79]*/*TEXT*.json"))
print(f"Number of files in dataset : {len(data_files):,}")

Number of files in dataset : 2,460


데이터 속성값들

In [7]:
with open(data_files[0], "r") as file:
    documents = json.load(file)["paragraphs"]
pprint(documents[:3])

[{'id': 'BOOK_CORPUS_190.560001',
  'info': {'author': {'birth_year': 1972,
                      'jobs': ['연구원', '작가'],
                      'write_age': 38},
           'class': 0,
           'kdc': '199',
           'published_year': 2010},
  'sentences': [{'char_count': 36,
                 'id': 'BOOK_CORPUS_190.560001.1',
                 'noise_ratio': 0.0,
                 'original_text': '“실질적으로 ‘긍정’으로 가는 첫 단계는 바로 ‘정의하기’일세.”',
                 'text': '“실질적으로 ‘긍정’으로 가는 첫 단계는 바로 ‘정의하기’일세.”',
                 'word_count': 7}]},
 {'id': 'BOOK_CORPUS_190.560002',
  'info': {'author': {'birth_year': 1967,
                      'jobs': ['번역가', '작가'],
                      'write_age': 48},
           'class': 0,
           'kdc': '199',
           'published_year': 2015},
  'sentences': [{'char_count': 51,
                 'id': 'BOOK_CORPUS_190.560002.1',
                 'noise_ratio': 0.0,
                 'original_text': '다시 얼마 후 개사왕은 궁궐 밖으로 나갔다가 여러 백성이 악기를 만드는 모습을 '
       

전체 로드. 

1. 만약 한 paragraphs 안에 여러 문장이 있으면 이를 공백을 경계로 이어붙인다.
2. 10자 미만인 문장은 수집하지 않는다.
3. 전체 데이터셋 용량이 꽤 크므로 파티션으로 분할

In [8]:
corpus = []
partition_idx = 0
for data_file in tqdm(data_files):
    with open(data_file, "r") as file:
        paragraphs = json.load(file)["paragraphs"]
    for paragraph in paragraphs:
        sentences = [sentence["text"] for sentence in paragraph["sentences"]]  # (1a)
        document = " ".join(sentences)
        if len(document) < 10:
            continue  # (2)
        else:
            corpus.append(document)
        if len(corpus) >= 1_000_000:  # (3)
            partition = pl.DataFrame({"document": corpus})
            partition.write_parquet(result_dir.joinpath(f"partition_{partition_idx:02}.parquet"))
            partition_idx += 1
            corpus = []
if corpus:
    partition = pl.DataFrame({"document": corpus})
    partition.write_parquet(result_dir.joinpath(f"partition_{partition_idx:02}.parquet"))

100%|███████████████████████████████████████| 2460/2460 [05:05<00:00,  8.06it/s]


### 문서 토큰 수 분포 확인

한국어에서는 정확히 적용되지 않지만, 편의를 위해 어절의 수(`공백 수 + 1`)를 토큰 수라고 정의한다. 문학 카테고리를 제외하고 수집해도 7억5천만개가 넘는 토큰을 확보할 수 있다.

In [9]:
num_tokens = []
for partition_path in tqdm(result_dir.glob("*.parquet")):
    partition = pl.read_parquet(partition_path)
    document_tokens = (
        partition
        .with_columns((pl.col("document").str.count_matches(" ") + 1).alias("num_tokens"))
        .to_dict(as_series=False)["num_tokens"]
    )
    num_tokens.extend(document_tokens)
print(f"Number of tokens in the corpus : {sum(num_tokens):,}")

22it [00:24,  1.10s/it]

Number of tokens in the corpus : 755,240,531


문서당 토큰 수 분포. 이상치가 포함되어 있음을 알 수 있다.

In [10]:
pl.DataFrame({"num_tokens": num_tokens}).select(
    pl.col("num_tokens").min().alias("min"),
    pl.col("num_tokens").quantile(0.05).cast(pl.Int32).alias("q05"),
    pl.col("num_tokens").quantile(0.25).cast(pl.Int32).alias("Q1"),
    pl.col("num_tokens").quantile(0.50).cast(pl.Int32).alias("median"),
    pl.col("num_tokens").quantile(0.75).cast(pl.Int32).alias("Q3"),
    pl.col("num_tokens").quantile(0.95).cast(pl.Int32).alias("q95"),
    pl.col("num_tokens").max().alias("max"),
)

min,q05,Q1,median,Q3,q95,max
i64,i32,i32,i32,i32,i32,i64
1,3,8,24,49,98,"144,808"
